# Initial setup

In [ ]:
# to print all output for a cell instead of only last one 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
#import all libraries and modules 

import os

import requests
import zipfile
import random
import string 

import torch

import numpy as np
import pandas as pd

import gensim
import gensim.downloader as gloader
from gensim.models import KeyedVectors

import time 
import logging
import tqdm

from collections import OrderedDict, namedtuple

import json
from pandas import json_normalize
	
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
print("Current work directory: {}".format(os.getcwd())) #print the current working directory 

data_folder = os.path.join(os.getcwd(),"data") # directory containing the notebook

if not os.path.exists(data_folder):   #create folder where all data will be stored 
    os.makedirs(data_folder)

In [ ]:
# Fix data seed to achieve reproducible results
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

#setup logging 
log = logging.getLogger('logger')
log.setLevel(logging.DEBUG)
fh = logging.FileHandler('data/log.txt')
fh.setLevel(logging.DEBUG)
log.addHandler(fh)

In [ ]:
json_file_path_train = "data/train.json"
json_file_path_val = "data/dev.json"
json_file_path_test = "data/test.json"

with open(json_file_path_train, 'r') as tr:
     contents = json.loads(tr.read())
     train = json_normalize(contents)

with open(json_file_path_val, 'r') as vl:
     contents = json.loads(vl.read())
     val = json_normalize(contents) 

with open(json_file_path_test, 'r') as ts:
     contents = json.loads(ts.read())
     test = json_normalize(contents) 

# Dataset inspection

In [ ]:
train = train[:4000]
val = val[:2000]
test = test[:500]

train.head(3)

In [ ]:
print(f"Training set size: {len(train)}")
print(f"Validation set size: {len(val)}")
print(f"Test set size: {len(test)}")

In [ ]:
print(f"N° of features per account: {len(train.columns)}")
print(f"Features:\n{list(train.columns)}") #TODO: rimuovere tutto ciò che è prima del punto e rinominare le colonne?

In [ ]:
print(f"N° of accounts without tweets: {train['tweet'].isna().sum()}")
print(f"N° of tweets per account analytics:\n{np.round(train['tweet'].dropna().apply(len)).describe()}")


# Tweets preprocessing

In [ ]:
import unidecode
import nltk
import emoji
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords

nltk.download('wordnet', quiet=True) 
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

def lemmatize_and_remove_non_ascii(sentence:str):
    """Remove unnecessary spaces, remove words with non ASCII characters and lemmatize"""
    lemmatizer = WordNetLemmatizer()
    sentence = [lemmatizer.lemmatize(word) for word in sentence if word.isascii()] #if a word has all ASCII characters: lemmatize, else: remove
    return sentence

def stemm_and_remove_non_ascii(sentence: str):
    ps = PorterStemmer()
    sentence = [ps.stem(word) for word in sentence if word.isascii()]#if a word has all ASCII characters: stemm, else: remove
    return sentence

def preprocess_pipeline(sentence:str):
    """Apply standard preprocessing"""

    #put everything to lowercase
    sentence = sentence.lower()

    #remove all unnecessary spaces and return a list of words
    sentence = sentence.split()

    return sentence

def preprocess_type1(sentence:str):
    """Apply standard preprocessing and return a list of words"""

    sentence = preprocess_pipeline(sentence)

    #TODO: decidere con cosa sostituire i caratteri speciali (@, link, RT, emoticons...)
    #replace link string with 'https'
    sentence = ['https' if 'http' in word else word for word in sentence]

    #remove emoticons
    sentence = [word for word in sentence if not emoji.is_emoji(word)]

    #remove all punctuation
    sentence = [word.translate(str.maketrans(dict.fromkeys(string.punctuation,''))) for word in sentence]

    return sentence

def preprocess_type2(sentence:str):
    """Apply standard preprocessing and return a list of words"""

    sentence = preprocess_pipeline(sentence)

    #TODO: decidere con cosa sostituire i caratteri speciali (@, link, RT, emoticons...)
    #replace link string with 'https'
    sentence = ['https' if 'http' in word else word for word in sentence]

    #replace emoticons with their descriptions
    sentence = [emoji.demojize(word) if emoji.is_emoji(word) else word for word in sentence]
    
    #transliterates any UNICODE string into the closest possible representation in ASCII text
    sentence = [unidecode.unidecode(word) for word in sentence]

    #remove all punctuation
    sentence = [word.translate(str.maketrans(dict.fromkeys(string.punctuation,''))) for word in sentence]

    sentence = [word for word in sentence if word != '']

    return sentence

def preprocess_type3(sentence:str):
    """Apply standard preprocessing, transliterates UNICODE characters in ASCII, 
    remove words with non ASCII characters, lemmatize and return a list of words"""

    sentence = preprocess_pipeline(sentence)

    #remove non-ascii words
    sentence = lemmatize_and_remove_non_ascii(sentence) #TODO: lemmatization doesn't work

    return sentence

def preprocess_type4(sentence: str):
    """
        Apply standard preprocessing, removes stop-words and non ascii's,  and stemmes.
    """
    sentence = preprocess_pipeline(sentence)
    stemmed = stemm_and_remove_non_ascii(sentence)
    stop_words = set(stopwords.words('english'))
    filter_stop_words = [word for word in stemmed if not word in stop_words]
    return filter_stop_words

In [ ]:
print(f"Original tweet:\n{train.loc[0, 'tweet'][0]}")
print(f"Processed tweet:\n{preprocess_type2(train.loc[0, 'tweet'][0])}") #TODO: le emoticon stanno dentro a GloVe?

In [ ]:
# Remove all accounts without tweets
train.dropna(subset=['tweet'], inplace=True)
val.dropna(subset=['tweet'], inplace=True)
test.dropna(subset=['tweet'], inplace=True)

train = train.explode('tweet').reset_index(drop=True)
val = val.explode('tweet').reset_index(drop=True)
test = test.explode('tweet').reset_index(drop=True)

# OLD VERSION vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
# # Preprocess all tweets
# def preprocess_tweets(data, preprocess_function):
#     for row in range(len(data)):
#         tweets = []
#         for tweet in data['tweet'].iloc[row]:
#             tweets.append(preprocess_function(tweet))
#         data.loc[:, 'tweet'].iloc[row] = tweets
#     return data

# train = preprocess_tweets(train, preprocess_type1)
# val = preprocess_tweets(val, preprocess_type1)
# test = preprocess_tweets(test, preprocess_type1)
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# Preprocess all tweets
train['tweet'] = train['tweet'].apply(preprocess_type2)
val['tweet'] = val['tweet'].apply(preprocess_type2)
test['tweet'] = test['tweet'].apply(preprocess_type2)

train = train[train['tweet'].map(lambda x: len(x)) > 0].reset_index(drop=True)
val = val[val['tweet'].map(lambda x: len(x)) > 0].reset_index(drop=True)
test = test[test['tweet'].map(lambda x: len(x)) > 0].reset_index(drop=True)

train.head(3)

# Vocabulary

In [ ]:
Vocab = namedtuple('Vocabulary',['word2int','int2word','unique_words'])

def build_vocab(unique_words : list[str]): 
    """
        Builds the dictionaries word2int, int2word and put them in the Vocabulary
    """
    word2int = OrderedDict()
    int2word = OrderedDict()

    for i, word in enumerate(unique_words):
        word2int[word] = i+1           #plus 1 since the 0 will be used as tag token 
        int2word[i+1] = word
    
    return Vocab(word2int,int2word,unique_words)

In [ ]:
unique_words_train = train['tweet'].explode().unique().tolist()
unique_words_val = val['tweet'].explode().unique().tolist()
unique_words_test = test['tweet'].explode().unique().tolist()

print('The number of unique words belonging to training set is:', len(unique_words_train))
print('The number of unique words belonging to validation set is:', len(unique_words_val))
print('The number of unique words belonging to test set is:', len(unique_words_test))

unique_words = set(unique_words_train + unique_words_val + unique_words_test)
print('The number of unique words in the entire dataset is:', len(unique_words))

In [ ]:
vocab = build_vocab(unique_words)

In [ ]:
def build_indexed_dataframe(df: pd.DataFrame):

    df['idx_tweet'] = df.tweet.apply(lambda x:list(map(vocab.word2int.get,x)))
    df['label'] = df.label.astype('category')   #convert the label column into category dtype
    df['label'] = df.label.cat.codes        #assign unique integer to each category
    df['ID'] = df['ID'].astype(np.int64)

    return df 

def check_dataframe_numberization(df,vocab):

    """
       Checks if the numberized dataframe will lead to the normal dataframe usind the reverse mapping 
    """

    tweet = df['tweet']

    idx_to_tweet = df.idx_tweet.apply(lambda x:list(map(vocab.int2word.get,x)))

    if tweet.equals(idx_to_tweet):
        print('CHECK COMPLETED: All right with dataset numberization')
    else:
        raise Exception('There are problems with Dataset numberization')

train = build_indexed_dataframe(train)
val = build_indexed_dataframe(val)
test = build_indexed_dataframe(test)

check_dataframe_numberization(train,vocab)

In [ ]:
train.head(3)

# Dataloaders

In [ ]:
from torchtext.data import BucketIterator
from torch.utils.data import Dataset

class DataframeDataset(Dataset):

    def __init__(self, dataframe: pd.DataFrame):

        dataframe = dataframe.copy()
        self.tweet = dataframe['idx_tweet']      #column of numberized tweets
        self.label = dataframe['label']       #column of categorical label 
        self.id = dataframe['ID']          #column of claim ids 

    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, idx):
        return {'tweet': self.tweet[idx],
                'label': self.label[idx],
                'ID': self.id[idx]}

def create_dataloaders(b_s : int, train: pd.DataFrame, val: pd.DataFrame, test: pd.DataFrame):     #b_s = batch_size

    #create DataframeDataset objects for each split 
    train_dataset = DataframeDataset(train)
    val_dataset = DataframeDataset(val)
    test_dataset = DataframeDataset(test)


    # Group similar length text sequences together in batches and return an iterator for each split.
    train_dataloader,val_dataloader,test_dataloader = BucketIterator.splits((train_dataset,val_dataset,test_dataset),
                                                        batch_sizes=(b_s,b_s,b_s), sort_key=lambda x: (len(x['tweet'])), 
                                                        repeat=True, sort=False, shuffle=True, sort_within_batch=True)
    
    return train_dataloader,val_dataloader,test_dataloader 

In [ ]:
temp_batch_size = 128
train_dataloader, val_dataloader, test_dataloader = create_dataloaders(temp_batch_size, train, val, test)
random_idx = random.randint(0, temp_batch_size-1)
train_dataloader.init_epoch()
for batch_id, batch in enumerate(train_dataloader.batches):
    print("Tweet: ", batch[random_idx]['tweet'])
    print("Label: ", batch[random_idx]['label'])
    print("Account Id: ", batch[random_idx]['ID'], "\n")
    print("Corresponding row in the dataset:")
    train[train['idx_tweet'].apply(lambda x: x == batch[random_idx]['tweet'])]
    break

# Word embeddings

In [ ]:
emb_matrix_path = os.path.join(data_folder, "emb_matrix.npy")
glove_model_path = os.path.join(data_folder, "glove_vectors.txt") 

def download_glove_emb(force_download = False):   
    """
        Download the glove embedding model and returns it 
    """
    emb_model = None

    if os.path.exists(glove_model_path) and not force_download: 
        print('glove vecotrs already saved in data folder, retrieving the file...')
        emb_model = KeyedVectors.load_word2vec_format(glove_model_path, binary=True)
        print('vectors loaded')

    else:
        print('downloading glove embeddings...')        
        embedding_dimension=300

        download_path = "glove-wiki-gigaword-{}".format(embedding_dimension)
        emb_model = gloader.load(download_path)

        print('saving glove embeddings to file')  
        emb_model.save_word2vec_format(glove_model_path, binary=True)
        
    return emb_model

force_download = False      # to download glove model even if the vectors model has been already stored. Mainly for testing purposes

glove_embeddings = download_glove_emb(force_download)

In [ ]:
def check_OOV_terms(embedding_model: gensim.models.keyedvectors.KeyedVectors, vocab):
    """
        Given the embedding model and the unique words in the dataframe, determines the out-of-vocabulary words 
    """
    oov_words = []
    idx_oov_words = []

    if embedding_model is None:
        print('WARNING: empty embeddings model')

    else: 
        for word in vocab.unique_words:
            try: 
                embedding_model[word]
            except:
                oov_words.append(word) 
                idx_oov_words.append(vocab.word2int[word]) 
        
        print("Total number of unique words in dataset:",len(vocab.unique_words))
        print("Total OOV terms: {0} ({1:.2f}%)".format(len(oov_words), (float(len(oov_words)) / len(vocab.unique_words))*100))
        print("Some OOV terms:",random.sample(oov_words,10))
    
    return oov_words, idx_oov_words

oov_words, idx_oov_words = check_OOV_terms(glove_embeddings,vocab)

In [ ]:
def build_embedding_matrix(emb_model: gensim.models.keyedvectors.KeyedVectors,vocab) -> np.ndarray:
    """
        If the embedding for the word is present, add it to the embedding_matrix, otherwise insert a vector of random values.
        Return the embedding matrix
    """
    if emb_model is None:
        print('WARNING: empty embeddings model')
        return None
    
    print('Building embedding matrix...')

    embedding_dimension = len(emb_model[0]) #how many numbers each emb vector is composed of                                                           
    embedding_matrix = np.zeros((len(vocab.word2int)+1, embedding_dimension), dtype=np.float32)   #create a matrix initialized with all zeros 

    for word, idx in vocab.word2int.items():
        try:
            embedding_vector = emb_model[word]
        except (KeyError, TypeError):
            embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)

        embedding_matrix[idx] = embedding_vector     #assign the retrived or the generated vector to the corresponding index 
    
    print(f"Embedding matrix shape: {embedding_matrix.shape}")

    return embedding_matrix

embedding_matrix = build_embedding_matrix(glove_embeddings, vocab)

In [ ]:
pd.DataFrame(embedding_matrix).head()

In [ ]:
def check_id_corr(glove: gensim.models.keyedvectors.KeyedVectors, vocab, matrix, dataframe):
    """
        Checks whether the numberized dataframe and the index of the embedding matrix correspond
    """
    if not glove:
        print('WARNING: empty model, remember to download GloVe first or set force_dowload to True')
        return 
    oov_words_ = []

    for indexed_sentence in dataframe['idx_tweet']:

        for token in indexed_sentence:
            embedding = matrix[token]
            word = vocab.int2word[token]
            if word in glove.key_to_index:
                assert(np.array_equal(embedding,glove[word]))
            else:
                oov_words_.append(word)

    print('Double check OOV number:',len(set(oov_words_)))

check_id_corr(glove_embeddings,vocab,embedding_matrix,train)
check_id_corr(glove_embeddings,vocab,embedding_matrix,val)
check_id_corr(glove_embeddings,vocab,embedding_matrix,test)

# Model design

In [ ]:
#pytorch imports

import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.rnn as rnn
import torch.nn.functional as F

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

#scikit-learn imports 
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
class Custom_model(nn.Module):
    """
        Class defining our model architecture  
    """
    def __init__(self, emb_matrix: np.ndarray, model_param : dict, device) :
        super().__init__()

        self.device = device
        self.model_param = model_param

        self.embedding_layer, self.word_embedding_dim = self.build_emb_layer(emb_matrix,model_param['pad_idx'], model_param['freeze_embedding'])

        self.rnn = nn.LSTM(self.word_embedding_dim, self.word_embedding_dim, batch_first = True) 
            
        self.drop_layer = nn.Dropout(p=0.5) 

        self.classifier = nn.Linear(self.word_embedding_dim,1)   

        self.to(self.device)  #move model to device , 'gpu' if possible 

    def build_emb_layer(self, weights_matrix: np.ndarray, pad_idx : int, freeze = True):
    
        matrix = torch.Tensor(weights_matrix).to(self.device)   #the embedding matrix 
        _ , embedding_dim = matrix.shape 

        emb_layer = nn.Embedding.from_pretrained(matrix, freeze=freeze, padding_idx = pad_idx)   #load pretrained weights in the layer and make it non-trainable (TODO: trainable ? )
        
        return emb_layer, embedding_dim
        

    def pad_batch(self,batch: list):    #pad each sentece of a batch to a common length
        """
            Input:  List of Tensors of variable length
            Output: Batch of tensors all padded to the same length 
        """
        batch = batch.copy() 

        padded_batch = rnn.pad_sequence(batch,batch_first = True, padding_value = self.model_param['pad_idx'])

        padded_batch = padded_batch.to(self.device)    #move tensor to gpu if possible 

        return padded_batch


    def words_embedding(self, word_idxs):   #get embedding vectors for each token in sentence 
        """
            Input:  [batch_size, num_tokens]
            Output: [batch_size, num_tokens, embedding_dim]
        """
        return self.embedding_layer(word_idxs)
    
    def sentence_embedding(self, embeddings, sentence_lenghts):     #compute sentence embedding 
        """
            Input:  [batch_size, num_tokens, embedding_dim]
            Output: [batch_size, embedding_dim]
        """
        #take as sentence embedding the average of all the states of the rnn corresponing to each word 

        packed_embeddings = pack_padded_sequence(embeddings, sentence_lenghts.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, _  = self.rnn(packed_embeddings)

        unpacked_out, l = pad_packed_sequence(packed_out,batch_first=True)

        avg = unpacked_out.sum(dim=1).div(sentence_lenghts.unsqueeze(dim=1))

        return avg

    def forward(self, tweet, tweet_lengths):

        #pad the sentences to have fixed size 
        padded_tweet = self.pad_batch(tweet)
        
        #embed each word in a sentence with a 300d vector 
        word_emb_tweet = self.words_embedding(padded_tweet)

        #compute sentence embedding
        sentence_emb_tweet = self.sentence_embedding(word_emb_tweet,tweet_lengths)

        #eventual dropout 
        if self.model_param['dropout']: 
            sentence_emb_tweet = self.drop_layer(sentence_emb_tweet)

        #final classification 
        predictions = self.classifier(sentence_emb_tweet)

        predictions = predictions.squeeze()   #remove dim of size 1 

        return predictions

In [ ]:
#compute accuracy and f1-score 
def acc_and_f1(y_true: torch.LongTensor,y_pred: torch.LongTensor):
    """
        Compute accuracy and f1-score for an epoch 
    """
    acc = accuracy_score(y_true, y_pred)

    f1 = f1_score(y_true,y_pred,average='macro')

    return acc,f1

In [ ]:
def train_loop(model: Custom_model, iterator : BucketIterator, optimizer: optim.Optimizer, criterion, device):
    """ Args:
         - model: the model istantiated with pre-defined hyperparameters.
         - iterator: dataloader for passing data to the network in batches 
         - optimizer: optimizer for backward pass 
         - criterion: loss function 
         - device: 'gpu' if it's available, 'cpu' otherwise 
    """
    start = time.perf_counter()

    batch_loss = 0
    
    #aggregate all the predictions and corresponding true labels (and claim ids) in tensors 
    all_pred , all_targ, all_ids = torch.LongTensor(), torch.LongTensor(), torch.LongTensor()

    model.train()
    
    iterator.init_epoch()  #generate and shuffles batches from dataloader #TODO: create_batches 

    for batch_id, batch in enumerate(iterator.batches):

        tweet_batch = [torch.LongTensor(example['tweet']) for example in batch]    #list of tensors of words id for each sentence in a batch 

        tweet_lengths = torch.Tensor([len(example['tweet']) for example in batch])         #lenght of each claim sentence before padding 

        target_labels = torch.Tensor([example['label'] for example in batch])     #label of each example in a batch
        target_ids = torch.LongTensor([example['ID'] for example in batch])  #id of each claim in a batch 

        #move tensors to gpu if possible 
        tweet_lengths = tweet_lengths.to(device)
        target_labels = target_labels.to(device)    

        #zero the gradients 
        model.zero_grad(set_to_none=True)
        optimizer.zero_grad()            

        predictions = model(tweet_batch, tweet_lengths)   #generate predictions 

        loss = criterion(predictions, target_labels)      #compute the loss 


        pred = (predictions > 0.0 ).int().cpu()              #get class label 

        #concatenate the new tensors with the one computed in previous steps
        all_pred = torch.cat((all_pred,pred))          
        all_targ = torch.cat((all_targ,target_labels.long().cpu()))
        all_ids = torch.cat((all_ids,target_ids))

        #backward pass 
        loss.backward()
        optimizer.step()

        batch_loss += loss.item()    #accumulate batch loss 


    acc, f1 = acc_and_f1(all_targ,all_pred)

    loss = batch_loss/(batch_id+1)    #mean loss 

    end = time.perf_counter()
    log.debug('train epoch time: %s',end-start)

    return loss, acc, f1


def eval_loop(model: Custom_model, iterator: BucketIterator, criterion, device):
    """ Args:
         - model: the sequence pos tagger model istantiated with fixed hyperparameters.
         - iterator: dataloader for passing data to the network in batches 
         - criterion: loss function 
         - device: 'gpu' if it's available, 'cpu' otherwise 
    """
     
    start = time.perf_counter()

    batch_loss = 0
    
    all_pred , all_targ, all_ids = torch.LongTensor(), torch.LongTensor(), torch.LongTensor() 
    
    model.eval()   #model in eval mode 
    
    iterator.init_epoch()  #TODO create_batches 

    with torch.no_grad(): #without computing gradients since it is evaluation loop
    
        for batch_id, batch in enumerate(iterator.batches):
            
            tweet_batch = [torch.LongTensor(example['tweet']) for example in batch]    #list of tensors of words id for each sentence in a batch 

            tweet_lengths = torch.Tensor([len(example['tweet']) for example in batch])         #lenght of each claim sentence before padding 

            target_labels = torch.Tensor([example['label'] for example in batch])     #label of each example in a batch
            target_ids = torch.LongTensor([example['ID'] for example in batch])  #id of each claim in a batch 

            #move tensors to gpu if possible 
            tweet_lengths = tweet_lengths.to(device)
            target_labels = target_labels.to(device)    

            predictions = model(tweet_batch, tweet_lengths)   #generate predictions 

            loss = criterion(predictions, target_labels)      #compute the loss 

            pred = (predictions > 0.0 ).int().cpu()         #get class label 

            #concatenate the new tensors with the one computed in previous steps
            all_pred = torch.cat((all_pred,pred))          
            all_targ = torch.cat((all_targ,target_labels.long().cpu()))
            all_ids = torch.cat((all_ids,target_ids))

            batch_loss += loss.item()   #accumulate batch loss 
            
    acc, f1 = acc_and_f1(all_targ,all_pred)

    loss = batch_loss/(batch_id+1)   #mean loss 

    end = time.perf_counter()
    log.debug('eval epoch time: %s',end-start)

    return loss, acc, f1

In [ ]:
def train_and_eval(model: Custom_model, dataloaders: tuple[BucketIterator,...], param : dict(), device):
    """
        Runs the train and eval loop and keeps track of all the metrics of the training model 
    """
    best_f1, best_epoch = -1, -1   #init best f1 score 

    model_metrics = {
        "train_loss": [],
        "train_acc": [],
        "train_f1": [],
        "val_loss": [],
        "val_acc": [],
        "val_f1": [],
    }

    # criterion = nn.BCEWithLogitsLoss(pos_weight=param['weight_positive_class']).to(device)    #Binary CrossEntropy Loss that accept raw input and apply internally the sigmoid 
    criterion = nn.BCEWithLogitsLoss().to(device)    #Binary CrossEntropy Loss that accept raw input and apply internally the sigmoid 

    optimizer = optim.Adam(model.parameters(), lr=param['lr'],  weight_decay=param['weight_decay'])   #L2 regularization 

    train_dataloader, eval_dataloader = dataloaders   #unpack dataloaders 

    for epoch in range(param['n_epochs']): #epoch loop

        start_time = time.perf_counter()
        
        train_metrics = train_loop(model, train_dataloader, optimizer, criterion, device) #train
        eval_metrics = eval_loop(model, eval_dataloader, criterion, device) #eval
        
        end_time = time.perf_counter()

        tot_epoch_time = end_time-start_time          

        train_epoch_loss, train_epoch_acc, train_epoch_f1 = train_metrics
        eval_epoch_loss, eval_epoch_acc, eval_epoch_f1 = eval_metrics

        if eval_epoch_f1 >= best_f1:
            best_f1 = eval_epoch_f1
            best_epoch = epoch+1
            if not os.path.exists('models'):        
                os.makedirs('models')
            torch.save(model.state_dict(),f'models/baseline.pt')


        #log Train and Validation metrics
        model_metrics['train_loss'].append(train_epoch_loss)
        model_metrics['train_acc'].append(train_epoch_acc)
        model_metrics['train_f1'].append(train_epoch_f1)
        model_metrics['val_loss'].append(eval_epoch_loss)
        model_metrics['val_acc'].append(eval_epoch_acc)
        model_metrics['val_f1'].append(eval_epoch_f1)
       
        
        log.debug('Elapsed time for epoch %s : %s \n',epoch+1,tot_epoch_time)

        print(f'Epoch: {epoch+1:02} | Epoch Time: {tot_epoch_time:.4f}')
        print(f'\tTrain Loss: {train_epoch_loss:.3f} | Train Acc: {train_epoch_acc*100:.2f}% | Train F1: {train_epoch_f1:.2f}')
        print(f'\t Val. Loss: {eval_epoch_loss:.3f} | Val. Acc: {eval_epoch_acc*100:.2f}% | Val. F1: {eval_epoch_f1:.2f}')
    
    log.debug('Best Eval F1: %s, obtained at epoch: %s \n\n',best_f1,best_epoch)

    return model_metrics

In [ ]:
#PARAMETERS, HYPERPARAMETERS AND USEFUL OBJECTS 

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'running on {DEVICE}')

PAD_IDX = 0                     # pad index

#hyperparameters
BATCH_SIZE = 512                # number of sentences in each mini-batch
LR = 1e-3                       # learning rate 
N_EPOCHS = 10                   # number of epochs
WEIGHT_DECAY = 1e-5             # regularization

#model parameters
FREEZE = False                  # wheter to make the embedding layer trainable or not              
DROPOUT = True                  # wheter to use dropout layer or not  


#to counteract class imbalance 
# (human, bot) = train['label'].value_counts()    #number of supports and refutes in the train dataset 
# weight_positive_class = torch.Tensor([bot/human]).to(DEVICE)  #weight to give to positive class 

max_tokens = max(train.tweet.apply(len).max(), val.tweet.apply(len).max(), test.tweet.apply(len).max())  #max number of tokens in a sentence in the entire dataset 


#train pipeline parameters dictionary 
train_param = {
    'lr': LR,
    'n_epochs': N_EPOCHS,
    'weight_decay': WEIGHT_DECAY,
    # 'weight_positive_class': weight_positive_class
    }

#model parameters dictionary
model_param = {
    'pad_idx' : PAD_IDX,
    'max_tokens' : max_tokens,
    'freeze_embedding' : FREEZE,  
    'dropout' : DROPOUT
}

#create dataloaders 
train_dataloader,val_dataloader,test_dataloader = create_dataloaders(BATCH_SIZE, train, val, test)

In [ ]:
# for batch_id, batch in enumerate(train_dataloader.batches):
#     if 0 in torch.Tensor([len(example['tweet']) for example in batch]):
#         print([torch.LongTensor(example['tweet']) for example in batch])

In [ ]:
# clear gpu memory 
import gc
def clean_gpu_cache():
    gc.collect()
    torch.cuda.empty_cache()

all_models_metrics = {}

In [ ]:
clean_gpu_cache()
                  
model = Custom_model(embedding_matrix, model_param, DEVICE)

model_metrics = train_and_eval(model, (train_dataloader,val_dataloader), train_param, DEVICE)